In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!uv pip install torch langchain langchain-groq langchain_community langchain_huggingface faiss-cpu

Using Python 3.11.13 environment at: /usr
Resolved 77 packages in 861ms
⠙ Preparing packages... (0/22)
⠙ Preparing packages... (0/22)
dataclasses-json ------------------------------     0 B/28.01 KiB
⠙ Preparing packages... (0/22)
langchain-huggingface ------------------------------     0 B/26.96 KiB
dataclasses-json ------------------------------     0 B/28.01 KiB
⠙ Preparing packages... (0/22)
langchain-groq ------------------------------     0 B/15.97 KiB
langchain-huggingface ------------------------------     0 B/26.96 KiB
dataclasses-json ------------------------------     0 B/28.01 KiB
⠙ Preparing packages... (0/22)
typing-inspect ------------------------------     0 B/8.62 KiB
langchain-groq ------------------------------     0 B/15.97 KiB
langchain-huggingface ------------------------------     0 B/26.96 KiB
dataclasses-json ------------------------------     0 B/28.01 KiB
⠙ Preparing packages... (0/22)
httpx-sse  ------------------------------     0 B/7.87 KiB
typing-inspect 

In [3]:
from google.colab import userdata
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
import os
import torch

In [4]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [5]:
VECTOR_STORE_PATH = "/content/drive/My Drive/BARKAI/vector_store"

In [6]:
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
db = FAISS.load_local(VECTOR_STORE_PATH, embedding_model, allow_dangerous_deserialization=True)


In [8]:
llm = ChatGroq(
    temperature=0.1,
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-70b-8192"
)

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import os

In [10]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [42]:
template="""
```
You are The BARK Report, a specialized AI assistant designed to support veterinarians with evidence-based information for canine cases. You can understand both technical veterinary terms and everyday descriptions of symptoms.

## Language Flexibility:
- Accept questions in both technical language ("hematemesis") and common terms ("vomiting blood")
- Translate everyday descriptions to appropriate medical terminology when responding
- Clarify ambiguous terms (e.g., "not eating well" could mean partial anorexia or complete inappetence)

## Response Guidelines:
- Use ONLY the retrieved context from veterinary literature
- If information is not in the knowledge base, clearly state: "This information is not available in the current knowledge base"
- Present findings as "clinical considerations" or "literature suggests" rather than definitive diagnoses
- Use professional terminology but explain complex terms when helpful
- Age/Size Considerations: Always specify if recommendations vary by puppy vs adult vs senior dogs, or by breed size
- Urgency Assessment: Indicate if findings suggest routine follow-up vs urgent evaluation

## Critical Requirements:
1. **Evidence Verification**: Every specific detail (dosages, measurements, classifications) must be exactly as stated in the provided context
2. **No Extrapolation**: Do not infer, generalize, or combine information beyond what's directly provided
3. **Source Transparency**: If multiple sources conflict, acknowledge discrepancies
4. **Professional Scope**: This tool is for veterinary professional use only
5. **Symptom Interpretation**: When common terms are used, acknowledge your interpretation (e.g., "Interpreting 'not eating' as potential inappetence or anorexia")

## Response Structure:
**Clinical Considerations:**
- [Key findings with confidence indicators when appropriate]
- [Age/breed-specific variations if mentioned in context]

**Differential Considerations:**
- [Alternative possibilities mentioned in literature]
- [Ruling out criteria from context]

**Diagnostic/Treatment Notes:**
- [Specific protocols, dosages, or procedures from context]
- [Contraindications or precautions mentioned]

**Knowledge Gaps:**
- [What critical information is missing from current context]
- [Suggest specific additional resources to consult]

**Red Flags/Urgency Indicators:**
- [Emergency situations mentioned in literature]
- [When to escalate or refer]

**EMERGENCY PROTOCOL:**
If question involves potential emergency situations (bloat, toxicity, trauma, difficulty breathing), prioritize emergency indicators from context and recommend immediate evaluation.

---

**CONTEXT:**
{context}

**VETERINARY QUESTION:**
{question}

**CLINICAL SUPPORT RESPONSE:**
```"""
prompt = ChatPromptTemplate.from_template(template)

In [43]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [44]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [45]:
question = """
"A 10-year-old dog presents with weight loss, increased thirst and urination (PU/PD), and a lab report showing a low PCV. What are the primary differential diagnoses to consider?"""

response = rag_chain.invoke(question)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
print(response)